In [13]:
import pandas as pd, numpy as np
import itertools, requests, json

In [14]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')
lm=pd.read_excel('data/szekelyhon_locs_curated.xlsx')

In [15]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [16]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]
lm["uid"] = lm["title"].map(str) + lm["content"]

In [17]:
dc=dc.set_index('uid')
dm=dm.set_index('uid')
lm=lm.set_index('uid')

In [18]:
dmi=dm.loc[dc.index.dropna()]

In [19]:
df=dmi.merge(dc).drop_duplicates()

In [20]:
dl=df.merge(lm)

Working df ready. Now infer location.

In [21]:
to_geo=[i.split(',') for i in dl['actual_locs'].unique() if str(i)!='nan']

In [22]:
to_geo=[i.strip() for i in list(itertools.chain.from_iterable(to_geo))]

Load HU names

In [23]:
coords=json.loads(open('geo/huncoords.json','r').read())
megyek=json.loads(open('geo/megyek.json','r').read())

In [29]:
data=[]
header=['index']
for i in dl.iteritems():
    header.append(i[0])
header.append('geo_loc')
header.append('latitude')
header.append('longitude')
header.append('id')
for i in dl.T.iteritems():
    dummy=[i[0]]
    for j in range(len(i[1])):
        if j==(len(i[1])-1):
            dummy.append(i[1][j])
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dtime=str(dummy[1])[:10]
            if dtime=='nan': dtime=u'N/A' 
            if str(i[1][j])!='nan':    
                for k in i[1][j].split(','):
                    tk=k.strip().replace('[','').replace(']','').replace("'",'')
                    dummy[-4]=tk
                    if tk in coords:
                        dummy[-3]=coords[tk][0]
                        dummy[-2]=coords[tk][1]
                    else:
                        dummy[-3]=''
                        dummy[-2]=''
                    #dummy[-1]=str(dummy[-3])+' ('+str(dummy[0]+1)+')\n'+dtime
                    dummy[-1]=str(dummy[-4])+', '+dtime
                    data.append(list(dummy))                    
            else:
                dummy[-1]=u'N/A' +', '+dtime
                data.append(list(dummy))
        else: 
            dummy.append(i[1][j])

In [30]:
df=pd.DataFrame(data)
df.columns=header
df

,index,Unnamed: 0,date,hudate,link,image,category,icategory,tags,title,...,relevant,severity,deaths,duplicate,predicted_locs,actual_locs,geo_loc,latitude,longitude,id
0,0,0,2019-04-06,2019. április 06.,https://szekelyhon.ro/aktualis/medvetamadasok-...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Maros megye', 'medve']",Medvetámadások után helyszíni terepszemlét tar...,...,1,3,0,0,Szászrégen,Szászrégen,Szászrégen,46.7711,24.7008,"Szászrégen, 0"
1,1,1,2019-04-07,2019. április 07.,https://szekelyhon.ro/aktualis/halomra-olte-a-...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['medve', 'Udvarhelyszék']",Halomra ölte a birkákat a medve,...,1,2,14,0,NaN,Szentábrahám,Szentábrahám,46.3347,25.0091,"Szentábrahám, 1"
2,2,2,2019-04-17,2019. április 17.,https://szekelyhon.ro/aktualis/legalabb-tizeno...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['medve', 'Székelyudvarhely']",Legalább tizenöt medve jár be Székelyudvarhelyre,...,1,0,0,0,Székelyudvarhely,Székelyudvarhely,Székelyudvarhely,46.305,25.2926,"Székelyudvarhely, 2"
3,3,3,2019-05-02,2019. május 02.,https://szekelyhon.ro/aktualis/celkeresztbe-ke...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['medve', 'Udvarhelyszék']",Célkeresztbe kerülhet a Szentábrahámon juhokat...,...,1,0,0,0,Szentábrahám,Szentábrahám,Szentábrahám,46.3347,25.0091,"Szentábrahám, 3"
4,4,4,2019-05-02,2019. május 02.,https://szekelyhon.ro/aktualis/fenykeppel-nyom...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Marosszék', 'medve']","Fényképpel nyomatékosítják, csak óvatosan, mer...",...,1,1,0,0,"['Marosvásárhely', 'Marosvásárhely', 'Koronka'...","['Jedd', 'Koronka', 'Székelybós']",Jedd,46.549,24.629,"Jedd, 4"
5,4,4,2019-05-02,2019. május 02.,https://szekelyhon.ro/aktualis/fenykeppel-nyom...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Marosszék', 'medve']","Fényképpel nyomatékosítják, csak óvatosan, mer...",...,1,1,0,0,"['Marosvásárhely', 'Marosvásárhely', 'Koronka'...","['Jedd', 'Koronka', 'Székelybós']",Koronka,46.5206,24.6189,"Koronka, 4"
6,4,4,2019-05-02,2019. május 02.,https://szekelyhon.ro/aktualis/fenykeppel-nyom...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Marosszék', 'medve']","Fényképpel nyomatékosítják, csak óvatosan, mer...",...,1,1,0,0,"['Marosvásárhely', 'Marosvásárhely', 'Koronka'...","['Jedd', 'Koronka', 'Székelybós']",Székelybós,46.5314,24.6698,"Székelybós, 4"
7,5,5,2019-05-14,2019. május 14.,https://szekelyhon.ro/aktualis/korabbi-elohely...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,['Háromszék'],Korábbi élőhelyén engedték szabadon a tavaly e...,...,1,0,0,0,"['Sepsibükszád', 'Haraly', 'Haraly', 'Sepsibük...",Sepsibükszád,Sepsibükszád,46.0077,25.8411,"Sepsibükszád, 5"
8,6,6,2019-05-15,2019. május 15.,https://szekelyhon.ro/aktualis/medve-tamadt-eg...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Csíkszék', 'medve']",Medve támadt egy madarasi juhászra,...,1,3,0,0,Csíkmadaras,Csíkmadaras,Csíkmadaras,46.4932,25.7458,"Csíkmadaras, 6"
9,7,7,2019-05-15,2019. május 15.,https://szekelyhon.ro/aktualis/szurkulettel-jo...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['Marosszék', 'medve']",Szürkülettel jövő félelem: hárombocsos medvétő...,...,1,1,0,0,NaN,Marosszentanna,Marosszentanna,46.5814,24.5546,"Marosszentanna, 7"


Save data for loading in viz app

In [31]:
df['checked']=''
df['exact_loc']=0
df['source']='Székelyhon'

In [32]:
df=df[['checked',
    'date',
    'link',
    'image',
    'source',
    'title',
    'content',
    'relevant',
    'severity',
    'deaths',
    'duplicate',
    'geo_loc',
    'exact_loc',
    'latitude',
    'longitude']].set_index('checked')

In [33]:
#PowerBI exploration data
df.to_excel('data/szekelyhon_data.xlsx')